# Pankreopriver diabetes mellitus vs Diabetes Type 1 

## Beta diversity

### Libraries

In [ ]:
# pd.options.display.max_columns= 999

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

In [ ]:
from skbio.stats.ordination import pcoa

In [ ]:
from skbio.stats.distance import DistanceMatrix

In [ ]:
from skbio.stats.distance import _permanova

### Data loading

In [ ]:
df = pd.read_csv(
    "/data/projects/2024/Effenberger-Diabetes/data/PDM merged 3.0_modified.csv"
)

In [ ]:
sample_info = pd.read_csv(
    "/data/projects/2024/Effenberger-Diabetes/data/20011/20011_SampleInfo.csv"
)

In [ ]:
df.rename(columns={"Probennummer": "sample_information"}, inplace=True)

In [ ]:
sample_info.rename(columns={"SampleInformation": "sample_information"}, inplace=True)

In [ ]:
df = df.merge(
    sample_info[["sample_information", "IMGM ID", "Type"]],
    on="sample_information",
    how="left",
)

In [ ]:
file_path = "/data/projects/2024/Effenberger-Diabetes/out/nf_core_ampliseq_003/qiime2/barplot/level-6.csv"
df_tax = pd.read_csv(file_path)

In [ ]:
df_tax.set_index("index", inplace=True)

In [ ]:
exclude_cols = [
    "sample_information",
    "age",
    "KHK1",
    "KHK2",
    "CA1",
    "CA2",
    "HbA1C (DCCT/NGSP)1",
    "HbA1C (DCCT/NGSP)2",
    "Glukose1",
    "Glukose2",
    "BMI1",
    "BMI2",
    "Pankreatektomie",
    "HbA1C_diff",
    "Glukose_diff",
    "BMI_diff",
    "KHK_diff",
    "CA_diff",
]

df_tax_bacteria = df_tax.drop(columns=exclude_cols, errors="ignore")

threshold = 0.1

mean_abundance = df_tax_bacteria.mean(axis=0)

low_abundance_taxa = mean_abundance[mean_abundance < threshold].index

df_low_abundance = df_tax_bacteria[low_abundance_taxa]

high_abundance_taxa = mean_abundance[mean_abundance >= threshold].index

df_high_abundance = df_tax_bacteria[high_abundance_taxa]

In [ ]:
file_path = "/data/projects/2024/Effenberger-Diabetes/out/nf_core_ampliseq_003/qiime2/diversity/beta_diversity/bray_curtis_distance_matrix-condition/raw_data.tsv"
bray_curtis = pd.read_csv(file_path, sep="\t")
bray_curtis = bray_curtis.iloc[:, 1:]

In [ ]:
file_path = "/data/projects/2024/Effenberger-Diabetes/out/nf_core_ampliseq_003/qiime2/diversity/beta_diversity/jaccard_distance_matrix-condition/raw_data.tsv"
jaccard = pd.read_csv(file_path, sep="\t")
jaccard = jaccard.iloc[:, 1:]

In [ ]:
jaccard.head()

In [ ]:
bray_curtis.head()

### Data cleaning

#### Metadata contains clinical information

In [ ]:
metadata_cols = df[
    [
        "IMGM ID",
        "sample_information",
        "Type",
        "age",
        "KHK1",
        "KHK2",
        "CA1",
        "CA2",
        "HbA1C (DCCT/NGSP)1",
        "HbA1C (DCCT/NGSP)2",
        "Glukose1",
        "Glukose2",
        "BMI1",
        "BMI2",
        "Pankreatektomie",
        "sex",
        "Insulin1",
        "Insulin2",
        "MASLD1",
        "MASLD2",
        "nikotin",
    ]
]

In [ ]:
metadata = metadata_cols.dropna(subset=["IMGM ID"])

In [ ]:
metadata["HbA1C_diff"] = metadata.apply(
    lambda x: (
        "increase"
        if x["HbA1C (DCCT/NGSP)2"] - x["HbA1C (DCCT/NGSP)1"] > 0
        else "decrease"
    ),
    axis=1,
)
metadata["Glukose_diff"] = metadata.apply(
    lambda x: "increase" if x["Glukose2"] - x["Glukose1"] > 0 else "decrease", axis=1
)
metadata["BMI_diff"] = metadata.apply(
    lambda x: "increase" if x["BMI2"] - x["BMI1"] > 0 else "decrease", axis=1
)

In [ ]:
def categorize_diff(before, after):
    if after == "ja" and before == "nein":
        return "onset"
    elif after == "nein" and before == "nein":
        return "absent"
    elif after == "nein" and before == "ja":
        return "resolved"
    elif after == "ja" and before == "ja":
        return "persistent"
    else:
        return "unknown"


metadata["KHK_diff"] = metadata.apply(
    lambda x: categorize_diff(x["KHK1"], x["KHK2"]), axis=1
)
metadata["CA_diff"] = metadata.apply(
    lambda x: categorize_diff(x["CA1"], x["CA2"]), axis=1
)

In [ ]:
metadata.rename(columns={"IMGM ID": "id"}, inplace=True)

In [ ]:
metadata_k = metadata[metadata["sample_information"].str.contains("K", na=False)]
metadata_dm = metadata[metadata["sample_information"].str.match("DM", na=False)]
metadata_pdm = metadata[metadata["sample_information"].str.contains("PDM", na=False)]

In [ ]:
metadata_k = metadata_k.drop_duplicates(subset=["id"], keep="first")
metadata_dm = metadata_dm.drop_duplicates(subset=["id"], keep="first")
metadata_pdm = metadata_pdm.drop_duplicates(subset=["id"], keep="first")

In [ ]:
metadata = metadata.drop_duplicates(subset=["id"], keep="first")

#### Microbial data contains taxonomic information from QUIIME

In [ ]:
microbial_data = df_high_abundance.drop(columns=metadata_cols, errors="ignore")

In [ ]:
microbial_data["id"] = microbial_data.index

In [ ]:
def extract_species_name(taxonomy):
    """Extract the last part of a taxonomy string (genus name)."""
    return taxonomy.split(";")[-1].strip()

In [ ]:
microbial_data.rename(
    columns={col: extract_species_name(col) for col in microbial_data.columns},
    inplace=True,
)

In [ ]:
microbial_data.rename(columns={"index": "id"}, inplace=True)

In [ ]:
microbial_data.columns = microbial_data.columns.str.strip()
metadata.columns = metadata.columns.str.strip()

print("microbial_data columns:", microbial_data.columns)
print("metadata columns:", metadata.columns)

if "id" in microbial_data.columns and "id" in metadata.columns:
    microbial_data = microbial_data.merge(
        metadata[
            [
                "id",
                "Type",
                "sample_information",
                "age",
                "KHK1",
                "KHK2",
                "CA1",
                "CA2",
                "HbA1C (DCCT/NGSP)1",
                "HbA1C (DCCT/NGSP)2",
                "Glukose1",
                "Glukose2",
                "Pankreatektomie",
                "BMI1",
                "BMI2",
            ]
        ],
        on="id",
        how="left",
    )

else:
    print("'id' column not found in one or both DataFrames.")

In [ ]:
ordinal_map = {
    "nein": 0,
    "Teilresektion links": 1,
    "Teilresektion rechts": 2,
    "Resektion": 3,
}

microbial_data["Pankreatektomie_encoded"] = microbial_data["Pankreatektomie"].map(
    ordinal_map
)

In [ ]:
microbial_data_original = microbial_data.copy()

In [ ]:
microbial_data = microbial_data.drop(columns=["Pankreatektomie"])

In [ ]:
jaccard.head()

#### PCoA - Beta diversity - Jaccard distance 

In [ ]:
df = jaccard

In [ ]:
df["Group1"].replace("Diabetes mellitus Typ1", "DM", inplace=True)
df["Group1"].replace("pankreopriver Diabetes", "PDM", inplace=True)
df["Group1"].replace("Kontrolle", "K", inplace=True)
df["Group2"].replace("Diabetes mellitus Typ1", "DM", inplace=True)
df["Group2"].replace("pankreopriver Diabetes", "PDM", inplace=True)
df["Group2"].replace("Kontrolle", "K", inplace=True)

In [ ]:
jaccard["PairType"] = jaccard.apply(
    lambda row: (
        f"{row['Group1']} vs {row['Group2']}"
        if row["Group1"] <= row["Group2"]
        else f"{row['Group2']} vs {row['Group1']}"
    ),
    axis=1,
)

In [ ]:
import pandas as pd
from scipy.spatial.distance import squareform

long_df = jaccard.copy()
dist_matrix = pd.pivot_table(
    long_df, index="SubjectID1", columns="SubjectID2", values="Distance"
)

dist_matrix = dist_matrix.combine_first(dist_matrix.T).fillna(0)

In [ ]:
pcoa_results = pcoa(dist_matrix)
pcoa_df = pcoa_results.samples

In [ ]:
pcoa_df.index = dist_matrix.index

In [ ]:
pcoa_df["id"] = pcoa_df.index

In [ ]:
pcoa_df = pcoa_df.merge(
    metadata[["id", "Type", "sample_information"]], on="id", how="left"
)

In [ ]:
pcoa_df["Type"] = pcoa_df["Type"].replace(
    {"Kontrolle": "K", "Diabetes mellitus Typ1": "DM", "pankreopriver Diabetes": "PDM"}
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D


pcoa_df["Type"] = pcoa_df["Type"].replace({
    "K": "H",
    "DM": "T1DM",
    "PDM": "T3cDM"
})


custom_palette = {
    "H": "#3374A1",
    "T1DM": "#E1812C",
    "T3cDM": "#3A923A"
}

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection="3d")

for group in pcoa_df["Type"].unique():
    subset = pcoa_df[pcoa_df["Type"] == group]
    ax.scatter(
        subset["PC1"],
        subset["PC2"],
        subset["PC3"],
        label=group,
        color=custom_palette.get(group, "gray"),
        s=50,
        alpha=0.8,
    )

ax.set_xlabel(f"PC1 ({pcoa_results.proportion_explained[0]*100:.2f}%)")
ax.set_ylabel(f"PC2 ({pcoa_results.proportion_explained[1]*100:.2f}%)")
ax.set_zlabel(f"PC3 ({pcoa_results.proportion_explained[2]*100:.2f}%)")
ax.set_title("")

ax.legend(title="Type")
plt.tight_layout()


#plt.savefig("/data/scratch/kvalem/projects/2024/diabetes_microbe/05-results/figures/jaccard_pcoa_3d.svg", dpi=300)
#plt.savefig("/data/scratch/kvalem/projects/2024/diabetes_microbe/05-results/figures/jaccard_pcoa_3d.png", dpi=300)

plt.show()


#### PCoA - Beta diversity - Bray Curtis distance 

In [ ]:
bray_curtis["PairType"] = bray_curtis.apply(
    lambda row: (
        f"{row['Group1']} vs {row['Group2']}"
        if row["Group1"] <= row["Group2"]
        else f"{row['Group2']} vs {row['Group1']}"
    ),
    axis=1,
)

In [ ]:
import pandas as pd
from scipy.spatial.distance import squareform

long_df = bray_curtis.copy()
dist_matrix = pd.pivot_table(
    long_df, index="SubjectID1", columns="SubjectID2", values="Distance"
)

dist_matrix = dist_matrix.combine_first(dist_matrix.T).fillna(0)

In [ ]:
pcoa_results = pcoa(dist_matrix)
pcoa_df = pcoa_results.samples

In [ ]:
pcoa_df.index = dist_matrix.index

In [ ]:
pcoa_df["id"] = pcoa_df.index

In [ ]:
pcoa_df = pcoa_df.merge(
    metadata[["id", "Type", "sample_information"]], on="id", how="left"
)

In [ ]:
pcoa_df["Type"] = pcoa_df["Type"].replace(
    {"Kontrolle": "K", "Diabetes mellitus Typ1": "DM", "pankreopriver Diabetes": "PDM"}
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D


pcoa_df["Type"] = pcoa_df["Type"].replace({
    "K": "H",
    "DM": "T1DM",
    "PDM": "T3cDM"
})


custom_palette = {
    "H": "#3374A1",
    "T1DM": "#E1812C",
    "T3cDM": "#3A923A"
}


fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection="3d")

for group in pcoa_df["Type"].unique():
    subset = pcoa_df[pcoa_df["Type"] == group]
    ax.scatter(
        subset["PC1"],
        subset["PC2"],
        subset["PC3"],
        label=group,
        color=custom_palette.get(group, "gray"),
        s=50,
        alpha=0.8,
    )

ax.set_xlabel(f"PC1 ({pcoa_results.proportion_explained[0]*100:.2f}%)")
ax.set_ylabel(f"PC2 ({pcoa_results.proportion_explained[1]*100:.2f}%)")
ax.set_zlabel(f"PC3 ({pcoa_results.proportion_explained[2]*100:.2f}%)")
ax.set_title("")


plt.savefig("/data/scratch/kvalem/projects/2024/diabetes_microbe/05-results/figures/bray_curtis_pcoa_3d.svg", dpi = 300)
plt.savefig("/data/scratch/kvalem/projects/2024/diabetes_microbe/05-results/figures/bray_curtis_pcoa_3d.png",dpi = 300)

# Add legend and layout
ax.legend(title="Type")
plt.tight_layout()
plt.show()